## **Installing Dependancies**

In [4]:
!pip install --upgrade pip
!pip install --upgrade setuptools
!pip install "folium>=0.8.2"
!pip install geopandas

Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (19.0.3)
Requirement already up-to-date: setuptools in /usr/local/lib/python3.6/dist-packages (40.8.0)


##**Importing Modules and Setting Up Data**

In [7]:
import pandas as pd
import geopandas as gpd
import folium
import datetime

from folium.plugins import HeatMap
from shapely.geometry import Point
from statistics import mean
from math import isnan
from base64 import b64decode


df = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vTCNvv5jcNGXYV9WdgP07OjkdEMk8jW5HpzUq67mwD7blNLbaDmt_Bl5LUX1aWbdXP7Zz9AmksrkafT/pub?gid=0&single=true&output=csv")
df["Date Listed"] = df["Date Listed"].apply(lambda x: datetime.datetime.strptime(x.strip() , "%d/%m/%Y"))
df["Listing Age"] = df["Date Listed"].apply(lambda x: (datetime.datetime.now() - x).total_seconds() // 86400)
df["geometry"] = [Point(x) for x in zip(df["Lat"] , df["Lng"])]
df = gpd.GeoDataFrame(df , crs={"init" : "epsg:4326"} , geometry="geometry")

df

,Name,ImageBase64Encoded,Date Listed,Lat,Lng,City,Listing Age,geometry
0,Alberta Mill Complex,NaN,1976-01-19,35.914167,-79.071389,Carrboro,15754.0,POINT (35.91416667 -79.07138888999999)
1,Ayr Mount,/9j/4AAQSkZJRgABAQEAAAAAAAD//gCGRmlsZSBzb3VyY2...,1971-08-26,36.076944,-79.091667,Hillsborough,17361.0,POINT (36.07694444 -79.09166667)
2,Bellevue Manufacturing Company,/9j/4AAQSkZJRgABAQEBFgEWAAD//gBYRmlsZSBzb3VyY2...,2003-08-28,36.073056,-79.111389,Hillsborough,5671.0,POINT (36.07305556 -79.11138889)
3,Beta Theta Pi Fraternity House,/9j/4AAQSkZJRgABAQEBFgEWAAD//gB0RmlsZSBzb3VyY2...,2005-04-20,35.913611,-79.055556,Chapel Hill,5070.0,POINT (35.91361111 -79.05555556)
4,Bingham School,/9j/4AAQSkZJRgABAQEBFgEWAAD//gBdRmlsZSBzb3VyY2...,1978-01-18,35.953611,-79.254444,Oaks,15024.0,POINT (35.95361111 -79.25444444)
5,Burwell School,/9j/4AAQSkZJRgABAQEBFgEWAAD//gBlRmlsZSBzb3VyY2...,1970-09-15,36.079167,-79.100000,Hillsborough,17706.0,POINT (36.07916667 -79.09999999999999)
6,Cabe-Pratt-Harris House,NaN,1999-04-22,36.058056,-79.007778,Hillsborough,7260.0,POINT (36.05805556 -79.00777778)
7,Carolina Inn,/9j/4AAQSkZJRgABAQEBLAEsAAD//gBGRmlsZSBzb3VyY2...,1999-08-06,35.910000,-79.055000,Chapel Hill,7154.0,POINT (35.91 -79.05500000000001)
8,Carrboro Commercial Historic District,/9j/4AAQSkZJRgABAQEBFgEWAAD//gBuRmlsZSBzb3VyY2...,1985-06-20,35.910278,-79.071667,Carrboro,12314.0,POINT (35.91027778 -79.07166667)
9,Cedar Grove Rural Crossroads Historic District,/9j/4AAQSkZJRgABAQEBFgEWAAD//gBoRmlsZSBzb3VyY2...,1998-04-23,36.170000,-79.169722,Cedar Grove,7624.0,POINT (36.17 -79.16972222)


##**Plotting Map**

In [6]:
#Create map
location = [mean([p.x for _ , p in df.geometry.iteritems()]) , mean([p.y for _ , p in df.geometry.iteritems()])]
m = folium.Map(location = location , zoom_start = 10.5)

#Create heat map
hmapData = [[point.x , point.y] for _ , point in df.geometry.iteritems()]
HeatMap(hmapData).add_to(m)

#Create markers
for _ , row in df.iterrows():
  point = row["geometry"]
  image = row["ImageBase64Encoded"]
  name = row["Name"]
  popup = name
  age = row["Listing Age"] - df["Listing Age"].min()
  
  #Add image if it exists
  if not(type(image) == float and isnan(image)):
    html = f'''
             <center>
             <p>{name}<br/></p>
             <img src="data:image/jpeg;base64,{image}" style="width:100px;height:75px;">
             </center>
           '''
    popup = folium.Popup(folium.IFrame(html , width=150 , height=150) , max_width=2650)
      
  folium.Marker([point.x , point.y , age] , popup=popup).add_to(m)
  

m.save("Map.html")
m